## WGAN CP (Wasserstein GAN)

201912203 이승찬

1.라이브러리 및 구글 드라이브 연결, 경로 설정

  WGAN CP

  - Wasserstein GAN 이며, 손실함수에 Wasserstein 거리를 이용하기 때문에 이렇게 불린다.
  - Discriminator가 Critic으로 바뀌는데, Real과 Fake를 구분하기 보다는 점수를 매기는 역할을 하기 때문이다.
  - Critic은 출력에 Sigmoid가 적용되어 있지 않아, 확률이 아닌 수를 출력하고 C_Real - C_Fake가 커지도록 강화된다.
  - WGAN의 NetC에 클리핑이 적용되어 있다.
  - 손실함수는 기존의 nn.BCE 에서 torch.mean(ouput)으로 바뀌었다.
  - Optimizer는 Adam 대신 RMSProp를 사용하게 바뀌었다.
  - 100 iter마다 진행상황을 출력하고, 1000 iter 마다영상을 저장한다.
  - batch_size = 256, learning_rate = 0.5e-4, epoch = 60, CilpValue = 0.005


In [57]:
# 구글 드라이브 연결

from google.colab import drive
drive.mount('/content/drive')

root = '/content/drive/MyDrive/Colab Notebooks/Gan기말과제/gan'

import os
import random
import torch
import torch.nn as nn   # 신경망
import torch.optim      # 최적화
from torch.utils.data import dataloader
from torchvision import datasets
from torchvision import transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
# Random Seed

seed = 100
random.seed(seed)
torch.manual_seed(seed)
torch.use_deterministic_algorithms(True)

# 하이퍼 파라미터

Hyper_Parameters = {

                     'num_workers' : 2,
                     'batch_size' : 256,
                     'image_size' : 64,
                     'input_channels' : 3,     # CIFAR10 은 컬러영상 : RGB채널 필요
                     'num_z' : 100,
                     'hidden_gen' : 64,
                     'hidden_disc' : 64,
                     'num_epoch' : 60,
                     'learning_rate' : 0.5e-4,
                     'ClipValue' : 0.005

}

2. Critic (Discriminator)
- Sigmoid가 가 없는 형태

In [59]:
class Critic(nn.Module): # 비평자 : 점수를 매김 fake => - real => +

    def __init__(self):
        super(Critic, self).__init__()

        self.disc = nn.Sequential(

                                    nn.Conv2d(
                                                 Hyper_Parameters['input_channels'], Hyper_Parameters['hidden_gen'],
                                                 kernel_size=4, stride=2, padding=1, bias=False),
                                                 nn.LeakyReLU(0.2),
                                                 nn.Conv2d(Hyper_Parameters['hidden_gen'], Hyper_Parameters['hidden_gen'] * 2,
                                                 kernel_size=4, stride=2, padding=1, bias=False),
                                                 nn.BatchNorm2d(Hyper_Parameters['hidden_gen'] * 2),
                                                 nn.LeakyReLU(0.2),
                                                 nn.Conv2d(Hyper_Parameters['hidden_gen'] * 2, Hyper_Parameters['hidden_gen'] * 4,
                                                 kernel_size=4, stride=2, padding=1, bias=False),
                                                 nn.BatchNorm2d(Hyper_Parameters['hidden_gen'] * 4),
                                                 nn.LeakyReLU(0.2),
                                                 nn.Conv2d(Hyper_Parameters['hidden_gen'] * 4, Hyper_Parameters['hidden_gen'] * 8,
                                                 kernel_size=4, stride=2, padding=1, bias=False),
                                                 nn.BatchNorm2d(Hyper_Parameters['hidden_gen'] * 8),
                                                 nn.LeakyReLU(0.2),
                                                 nn.Conv2d(Hyper_Parameters['hidden_gen'] * 8, 1,
                                                 kernel_size=4, stride=1, padding=0, bias=False)
            )

    def forward(self, input):
        return self.disc(input)

3. Generator

- 수업시간에 작성한 Generator를 사용

In [60]:
class Generator(nn.Module):

    def __init__(self):

        super(Generator, self).__init__()
        self.gen = nn.Sequential(

               nn.ConvTranspose2d(
                                     Hyper_Parameters['num_z'], Hyper_Parameters['hidden_gen'] * 8,
                                     kernel_size=4, stride=1, padding=0, bias=False),
                                     nn.BatchNorm2d(Hyper_Parameters['hidden_gen'] * 8),
                                     nn.ReLU(),
                                     nn.ConvTranspose2d(Hyper_Parameters['hidden_gen'] * 8, Hyper_Parameters['hidden_gen'] * 4,
                                     kernel_size=4, stride=2, padding=1, bias=False),
                                     nn.BatchNorm2d(Hyper_Parameters['hidden_gen'] * 4),
                                     nn.ReLU(),
                                     nn.ConvTranspose2d(Hyper_Parameters['hidden_gen'] * 4, Hyper_Parameters['hidden_gen'] * 2,
                                     kernel_size=4, stride=2, padding=1, bias=False),
                                     nn.BatchNorm2d(Hyper_Parameters['hidden_gen'] * 2),
                                     nn.ReLU(),
                                     nn.ConvTranspose2d(Hyper_Parameters['hidden_gen'] * 2, Hyper_Parameters['hidden_gen'],
                                     kernel_size=4, stride=2, padding=1, bias=False),
                                     nn.BatchNorm2d(Hyper_Parameters['hidden_gen']),
                                     nn.ReLU(),
                                     nn.ConvTranspose2d(Hyper_Parameters['hidden_gen'], Hyper_Parameters['input_channels'],
                                     kernel_size=4, stride=2, padding=1, bias=False),
                                     nn.Tanh()

            )

    def forward(self, input):

        return self.gen(input)

4. 데이터셋 및 신경망 초기화(CIFAR 사용)
- MNIST 사용 안함으로 삭제

In [61]:
dataset = datasets.CIFAR10(

        root=os.path.join(root, 'data'),
        download=True,
        transform=transforms.Compose([

            transforms.Resize(Hyper_Parameters['image_size']),
            transforms.CenterCrop(Hyper_Parameters['image_size']),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]

        ))

dataloader = torch.utils.data.DataLoader(

     dataset, batch_size = Hyper_Parameters['batch_size'],
     shuffle=True, num_workers = Hyper_Parameters['num_workers'])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Files already downloaded and verified


In [62]:
def weights_init(m):
    classname = m.__class__.__name__

    # 'ConvTranpose2d', 'Conv2d', 'ReLU', 'BatchNorm2d'

    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

netG = Generator().to(device)
netC = Critic().to(device)

netG.apply(weights_init)
netC.apply(weights_init)

Critic(
  (disc): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
  )
)

5. Otimizer & 노이즈 생성

- 기존의 Adam 대신 RMSprop을 사용한다.

In [63]:
real_label = 1.
fake_label = 0.

optimizerG = torch.optim.RMSprop( netG.parameters(), lr = Hyper_Parameters['learning_rate'] )
optimizerC = torch.optim.RMSprop( netC.parameters(), lr = Hyper_Parameters['learning_rate'] )

fixed_noise = torch.randn(64, Hyper_Parameters['num_z'], 1, 1, device=device)
print(fixed_noise.size())

torch.Size([64, 100, 1, 1])


6. 학습 시작

In [64]:
print("---------------------------------------------------------------------------------------------")
print("학습 시작:")
print("---------------------------------------------------------------------------------------------")

        # ============================================================= 시작

iters = 0

for epoch in range( Hyper_Parameters['num_epoch'] ):

    for i, data in enumerate(dataloader):

        # ============================================================= Critic_Real

        netC.zero_grad()
        real = data[0].to(device)
        b_size = real.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float,device=device)

        output = netC(real).view(-1)

        outC_real =  output.mean().item() # Real
        errC_real = -torch.mean(output) # WGAN 에서는 BSE대신 사용
        errC_real.backward() # 학습

        # outC_real => 양의 방향으로 가도록 학습

        # ============================================================= Critic_Fake

        noise = torch.randn(b_size, Hyper_Parameters['num_z'], 1, 1, device=device)
        fake = netG(noise)
        label.fill_(fake_label)

        output = netC(fake.detach()).view(-1)

        outC_fake =  output.mean().item()

        errC_fake = torch.mean(output) # 마찬가지로 BSE 대신 사용
        errC_fake.backward() # 학습을 위해 역전파해줌

        # =============================================================

        # outC_fake => 음의 방향으로 가도록 학습

        # outC_real - outC_fake가 증가하는 것은

        # Critic의 구별 능력이 상승하고 있음을 나타낸다.

        # ============================================================= Generator

        optimizerC.step()

        netG.zero_grad()
        label.fill_(real_label)

        output = netC(fake).view(-1)
        errG = -torch.mean(output)
        errG.backward()
        optimizerG.step()

        # ============================================================= WGAN Cliping 수행

        for p in netC.parameters():
          p.data.clamp_(-Hyper_Parameters['ClipValue'], Hyper_Parameters['ClipValue'])

        # ============================================================= 현재 상태 출력

        if iters % 100 == 0:

            Progress = int((float(epoch) / float(Hyper_Parameters['num_epoch'])) * 10)
            Bar = "■" * Progress + "□" * (10 - Progress)
            print(Bar + " | 현재 Epoch : %d | 총 Epoch : %d | 현재 iters : %d | " % (epoch, Hyper_Parameters['num_epoch'], iters))

        # ============================================================= 영상 1000 iter마다 저장

        if (iters % 1000 == 0): # 1000 iter 마다의 결과를 저장

            with torch.no_grad():

                fake = netG(fixed_noise).detach().cpu()

            plt.figure(figsize=(8, 8))  # 8x8
            plt.axis("off")
            plt.imshow(
                np.transpose(
                    vutils.make_grid(fake.to(device)[:64],
                                    padding=2, normalize=True).cpu(),
                                     (1, 2, 0)))

            frame_path = os.path.join(root, "Image{}.png".format(iters))
            plt.savefig(frame_path, bbox_inches='tight', pad_inches=0)
            plt.close()

        iters += 1

---------------------------------------------------------------------------------------------
학습 시작:
---------------------------------------------------------------------------------------------
□□□□□□□□□□ | 현재 Epoch : 0 | 총 Epoch : 60 | 현재 iters : 0 | 
□□□□□□□□□□ | 현재 Epoch : 0 | 총 Epoch : 60 | 현재 iters : 100 | 
□□□□□□□□□□ | 현재 Epoch : 1 | 총 Epoch : 60 | 현재 iters : 200 | 
□□□□□□□□□□ | 현재 Epoch : 1 | 총 Epoch : 60 | 현재 iters : 300 | 
□□□□□□□□□□ | 현재 Epoch : 2 | 총 Epoch : 60 | 현재 iters : 400 | 
□□□□□□□□□□ | 현재 Epoch : 2 | 총 Epoch : 60 | 현재 iters : 500 | 
□□□□□□□□□□ | 현재 Epoch : 3 | 총 Epoch : 60 | 현재 iters : 600 | 
□□□□□□□□□□ | 현재 Epoch : 3 | 총 Epoch : 60 | 현재 iters : 700 | 
□□□□□□□□□□ | 현재 Epoch : 4 | 총 Epoch : 60 | 현재 iters : 800 | 
□□□□□□□□□□ | 현재 Epoch : 4 | 총 Epoch : 60 | 현재 iters : 900 | 
□□□□□□□□□□ | 현재 Epoch : 5 | 총 Epoch : 60 | 현재 iters : 1000 | 
□□□□□□□□□□ | 현재 Epoch : 5 | 총 Epoch : 60 | 현재 iters : 1100 | 
■□□□□□□□□□ | 현재 Epoch : 6 | 총 Epoch : 60 | 현재 iters : 1200 | 
■□□□□□□□□□ 